# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env #Package tested on Python 3.6.8
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

# Setup Fiducial Cosmology

Import relevant python packages... 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

sns.set()

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Specify the values of your fiducial cosmology and other physical constants.

In [ ]:
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 0, 
        "z_pk" : 0.7} #Not necessary, but helps neatness later on

A_s = 2.2321e-9
n_s = 0.96659
omega_b = 0.02226
omega_cdm = 0.11271
tau_reio = 0.059888
h = 0.70148
z_pk = 0.7

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * fid['h'] #Units of m/s/Mpc
kp = 0.05 * fid['h'] #Units of 1/Mpc
t_cmb = 2.726


# Calculate Power Spectra

Specify points of interest to accumulate power spectra at... 

In [ ]:
z_table = np.arange(0.7, 2.1, 0.1)
m_chi_table = np.logspace(-1.0, 1.0, num=21, endpoint=True)

Determine derived quantities... Here, we assume the following relations: 

$$ \omega_\chi = \Big(\frac{T_\chi}{1.95 \textrm{[K]} }\Big)^3 \Big( \frac{m_\chi}{94 \textrm{[eV]}}\Big) $$

$$ \omega_m = \omega_b + \omega_{cdm} + \omega_\chi $$

$$ \omega_\lambda = h^2 - \omega_m$$

$$ V_{eff} = \Big(\frac{4 \pi}{3}\Big)\Big(\frac{c}{H}\Big)^3\Big( \int_{0.7}^{2.0} \frac{h}{\sqrt{\omega_m (1+z)^3 + \omega_\lambda}}  ~dz\Big)^3 $$

In [ ]:
T_chi = 0.5 #CAUTION! USING A SINGLE T VALUE TO CALCULATE THIS BLOCK!
omega_chi_table = np.power((T_chi / 1.95), 3.) * (m_chi_table / 94.)
omega_m_table = fid["omega_b"] + fid["omega_cdm"] + omega_chi_table
omega_lambda_table = np.power(h, 2.) - omega_m_table
v_eff_table = cf.v_effective_table_generator(z_table, omega_m_table, omega_lambda_table, c, h, H)
k_table = cf.k_table_generator(v_eff_table, h, 0.1, 100) 
prim_table = cf.primordial_table_generator(k_table, A_s, n_s, kp)

At this point, we have $\omega_m$, $\omega_{\lambda}$, $\omega_{\chi}$, $V_{eff}$ for several values of $m_\chi$ assuming a particular value of $T_\chi$. For transparency, let's show what these all look like... 

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 5))
ax[0].plot(m_chi_table, omega_chi_table)
ax[0].set_title(r'$\omega_\chi$')
ax[0].set_xlabel(r'$m_\chi$ [eV]')
ax[1].plot(m_chi_table, omega_lambda_table, label=r'$\omega_\lambda$')
ax[1].set_title(r'$\omega_\lambda$')
ax[1].set_xlabel(r'$m_\chi$ [eV]')
ax[2].plot(m_chi_table, omega_m_table, label=r'$\omega_m$')
ax[2].set_title(r'$\omega_m$')
ax[2].set_xlabel(r'$m_\chi$ [eV]')
ax[3].plot(m_chi_table, v_eff_table)
ax[3].set_title(r'$V_{eff}$ [Mpc$^3$]')
ax[3].set_xlabel(r'$m_\chi$ [eV]')
plt.show()

These all behave in an obvious way. Notice also that our k_table, which spans 100 points at each choice of $m_\chi$, has the first few values of k given by: 

In [ ]:
pd.DataFrame(k_table).head()

Define path where all data runs are stored. Anytime we require spectrum data from CLASS, the code will first check whether such data has already been calculated. If so, it will just import that data rather than re-run CLASS. We also specify a path to the CLASS install directory. 

In [ ]:
projectdir = cf.correct_path("~/Desktop/test/")
datastore = os.path.join(projectdir, "data")
classpath = os.path.join(projectdir, "class")

If you need to install CLASS, you can do so using the built-in function:

In [ ]:
#cf.install_class(projectdir)

At this point, we can quickly generate datasets of interest and plot results. For instance, consider a 1 eV relic with $T = 0.1, 0.2, 0.3, 0.4, 0.5$ [K]... 

In [ ]:
gen = []
spectra = []
for i in range(6): 
    sample = dict(fid)
    sample["m_ncdm"] = 1.0
    sample["N_ncdm"] = 1
    sample["T_ncdm"] = (i+1) * 0.1
    gen.append(sample)
    del sample
for samp in gen: 
    p = cf.generate_data(fid, classpath, datastore, **samp)
    spectra = np.append(spectra, cf.spectrum(p.replace('/test_parameters.ini','')))
for el in spectra: 
    el.interpolate(k_table[:,10]/h)
    el.generate_power_spectrum(omega_b, omega_cdm, k_table[:,10], prim_table[:,10])

In [ ]:
sns.set_palette("Blues_d", n_colors=len(spectra))
plt.figure(figsize=(10, 10))
for el in spectra: 
    plt.plot(k_table[:, 10], el.ps_table, label="T={0:.2f}".format(el.T_ncdm))
plt.xlabel("k [Mpc]")
plt.ylabel(r'$P_g$')
plt.title(r'Power spectra for $m_\chi=0.1$ eV, $z=0.7$, various $T_\chi$')
plt.legend()
plt.show()

# Convergence Tests


Working only the choice of $m_\chi = 0.1$ [eV], we wish to demonstrate convergence in some choice with which to vary the cosmological parameters of our model. To do so, we only need to specify our redshift binning and the variation on each parameter we would like to test. We repeat lines above for clarity. 

### Setup

In [ ]:
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 1, #Note this is wrong, but we leave it here to compare results to corrected version later
        "z_pk" : 0.7} #Not necessary, but helps neatness later on

A_s = 2.2321e-9
n_s = 0.96659
omega_b = 0.02226
omega_cdm = 0.11271
tau_reio = 0.059888
h = 0.70148
z_pk = 0.7

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc
t_cmb = 2.726

z_table = np.arange(0.7, 2.1, 0.1)
m_chi_table = np.array([1.0])

T_chi = 0.5 #Vary our relic temperature to 0.5 K 
omega_chi_table = np.power((T_chi / 1.95), 3.) * (m_chi_table / 94.)
omega_m_table = fid["omega_b"] + fid["omega_cdm"] + omega_chi_table
omega_lambda_table = np.power(h, 2.) - omega_m_table
v_eff_table = cf.v_effective_table_generator(z_table, omega_m_table, omega_lambda_table, c, h, H)
k_table = cf.k_table_generator(v_eff_table, h, 0.1, 100) 
prim_table = cf.primordial_table_generator(k_table, A_s, n_s, kp)

### Run Convergence Analysis

In [ ]:
relvary = np.arange(1.01, 1.11, 0.01)
t_vary = np.arange(0.1, 2.1, 0.1)

a1 = cf.analysis('a1', z_table, k_table[:,0], classpath, datastore)
a1.generate_fiducials(fid)
a1.generate_non_fiducials(fid, 
                          absvary_h=relvary*h,
                          absvary_omega_b=relvary*omega_b,
                          absvary_omega_cdm=relvary*omega_cdm,
                          absvary_tau_reio=relvary*tau_reio,
                          absvary_T_ncdm=t_vary)

Now calculate the derivatives... 

In [ ]:
for idx1, el1 in enumerate(a1.h_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*h, 'h')
for idx1, el1 in enumerate(a1.omega_b_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*omega_b, 'omega_b')
for idx1, el1 in enumerate(a1.omega_cdm_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*omega_cdm, 'omega_cdm')
for idx1, el1 in enumerate(a1.tau_reio_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*tau_reio, 'tau_reio')
for idx1, el1 in enumerate(a1.T_ncdm_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, t_vary[idx2], 'T_ncdm')

### Plot Power Spectra

First, let's just review what the power spectra look like for $m_\chi=1$ eV and $z=0.7$...

In [ ]:
sns.set_palette("Blues_d", n_colors=len(z_table))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.fid): 
    plt.plot(k_table, ps.ps_table, label='z={0:.1f}'.format(z_table[idx]))
plt.title(r'$P_g$ for $m_\chi=1$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

### Plot derivatives of power spectra

Now, we plot $\frac{dP_g}{d\theta}$ for $z=0.7$ in each parameter... 

In [ ]:
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.h_var[0]): 
    plt.plot(k_table, ps.dps_table['h'], label=r'$\delta$'+'h={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/dh$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_b_var[0]): 
    plt.plot(k_table, ps.dps_table['omega_b'], label=r'$\delta\omega_b=$'+'{0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\omega_b$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_cdm_var[0]): 
    plt.plot(k_table, ps.dps_table['omega_cdm'], label=r'$\delta\omega_{cdm}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\omega_{cdm}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.tau_reio_var[0]): 
    plt.plot(k_table, ps.dps_table['tau_reio'], label=r'$\delta\tau_{reio}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\tau_{reio}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(t_vary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.T_ncdm_var[0]): 
    plt.plot(k_table, ps.dps_table['T_ncdm'], label=r'$\delta T_{ncdm}=$'+'{0:.2f}'.format((t_vary[idx])))
plt.title(r'$dP_g/dT_{ncdm}$ for $z=0.7$, $m_\chi=1.0$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

### Plot difference between power spectra derivatives

In [ ]:
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.h_var[0])-1): 
    plt.plot(k_table, a1.h_var[0][i+1].dps_table['h'] - a1.h_var[0][i].dps_table['h'], 
             label=r'$\Delta$ h $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/dh)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_b_var[0])-1): 
    plt.plot(k_table, a1.omega_b_var[0][i+1].dps_table['omega_b'] - a1.omega_b_var[0][i].dps_table['omega_b'], 
             label=r'$\Delta \omega_b |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\omega_b)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_cdm_var[0])-1): 
    plt.plot(k_table, a1.omega_cdm_var[0][i+1].dps_table['omega_cdm'] - a1.omega_cdm_var[0][i].dps_table['omega_cdm'], 
             label=r'$\Delta \omega_cdm |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\omega_cdm)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.tau_reio_var[0])-1): 
    plt.plot(k_table, a1.tau_reio_var[0][i+1].dps_table['tau_reio'] - a1.tau_reio_var[0][i].dps_table['tau_reio'], 
             label=r'$\Delta \tau_{reio}$'+' $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\tau_{reio})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.T_ncdm_var[0])-1): 
    plt.plot(k_table, a1.T_ncdm_var[0][i+1].dps_table['T_ncdm'] - a1.T_ncdm_var[0][i].dps_table['T_ncdm'], 
             label=r'$\Delta T_{ncdm}$'+' $|^{:.1f}_{:.1f}$'.format(t_vary[i+1], t_vary[i]))
plt.title(r'$\Delta(dP_g/dT_{ncdm})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

### Convergence Tests 2

There are a few problems to note from our plots above... 

#### Problem 1: Too Difficult to See Differences without Plotting on Log Scale 

The low-k values of the power spectra contain significant portions of the data information. To better emphasize the features at low-k, we should plot the log of the power spectra, rather than just the power spectra. 

#### Problem 2: All Plots Look the Same

For example, we certainly don't expect changes in $\tau_{reio}$ to have effects on the power spectrum that are similar in scale to $\omega_b$. Our hunch is that we cannot leave N_ncdm=1 at all times, but must turn it on/off depending on whether we are calculating a dataset with/without an $\omega_\chi$ species. 

_Ok, it seems that upon first generating data the plots look fine. Still to see if the result is invariant after already having generated the data._

#### An attempt at resolving these problems... 



In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

sns.set()

import cosmicfish as cf

projectdir = cf.correct_path("~/Desktop/test/")
datastore = os.path.join(projectdir, "data")
classpath = os.path.join(projectdir, "class")

#Setup fiducial cosmology, note change to N_ncdm... 
fid = {"A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "N_ncdm" : 0, #Note the change! 
        "z_pk" : 0.7} #Not necessary, but helps neatness later on

A_s = 2.2321e-9
n_s = 0.96659
omega_b = 0.02226
omega_cdm = 0.11271
tau_reio = 0.059888
h = 0.70148
z_pk = 0.7

c = 2.9979e8 #Units of m/s
H = 1000. * 100. * h #Units of m/s/Mpc
kp = 0.05 * h #Units of 1/Mpc
t_cmb = 2.726

z_table = np.arange(0.7, 2.1, 0.1)
m_chi_table = np.array([1.0])


T_chi = 0.5 #Vary our relic temperature to 0.5 K 
omega_chi_table = np.power((T_chi / 1.95), 3.) * (m_chi_table / 94.)
omega_m_table = fid["omega_b"] + fid["omega_cdm"] + omega_chi_table
omega_lambda_table = np.power(h, 2.) - omega_m_table
v_eff_table = cf.v_effective_table_generator(z_table, omega_m_table, omega_lambda_table, c, h, H)
k_table = cf.k_table_generator(v_eff_table, h, 0.1, 100) 
prim_table = cf.primordial_table_generator(k_table, A_s, n_s, kp)

domega_chi_dT_chi = (3. * np.power(T_chi, 2.) * m_chi_table) / (np.power(1.95, 3.) * 94.)
dT_chi_domega_chi = (1./3.) *1.95 * np.power(( 94) / m_chi_table, 1./3.) * np.power(omega_chi_table, -2./3.)
print(1./domega_chi_dT_chi)
print(dT_chi_domega_chi)

In [ ]:
#Generate datasets
relvary = np.arange(1.01, 1.06, 0.01)
t_vary = np.arange(0.1, 0.6, 0.1)

a1 = cf.analysis('a1', z_table, k_table[:,0], classpath, datastore)
a1.generate_fiducials(fiducial=fid)
a1.generate_non_fiducials(fiducial=fid, 
                          absvary_h=relvary*h,
                          absvary_omega_b=relvary*omega_b,
                          absvary_omega_cdm=relvary*omega_cdm,
                          absvary_tau_reio=relvary*tau_reio,
                          absvary_T_ncdm=t_vary)

In [ ]:
#Calculate derivatives
for idx1, el1 in enumerate(a1.h_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*h, 'h')
for idx1, el1 in enumerate(a1.omega_b_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*omega_b, 'omega_b')
for idx1, el1 in enumerate(a1.omega_cdm_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*omega_cdm, 'omega_cdm')
for idx1, el1 in enumerate(a1.tau_reio_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, (1-relvary[idx2])*tau_reio, 'tau_reio')
for idx1, el1 in enumerate(a1.T_ncdm_var): 
    for idx2, el2 in enumerate(el1): 
        el2.dPs(a1.fid[idx1].ps_table, t_vary[idx2], 'T_ncdm')

In [ ]:
#Plot power spectra
sns.set_palette("Blues_d", n_colors=len(z_table))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.fid): 
    plt.plot(k_table, ps.ps_table, label='z={0:.1f}'.format(z_table[idx]))
plt.title(r'$P_g$ for $m_\chi=1$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

In [ ]:
#Plot derivatives of power spectra
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.h_var[0]): 
    plt.plot(k_table, ps.dps_table['h'], label=r'$\delta$'+'h={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/dh$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_b_var[0]): 
    plt.plot(k_table, ps.dps_table['omega_b'], label=r'$\delta\omega_b=$'+'{0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\omega_b$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_cdm_var[0]): 
    plt.plot(k_table, ps.dps_table['omega_cdm'], label=r'$\delta\omega_{cdm}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\omega_{cdm}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.tau_reio_var[0]): 
    plt.plot(k_table, ps.dps_table['tau_reio'], label=r'$\delta\tau_{reio}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dP_g/d\tau_{reio}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(t_vary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.T_ncdm_var[0]): 
    plt.plot(k_table, ps.dps_table['T_ncdm'], label=r'$\delta T_{ncdm}=$'+'{0:.2f}'.format((t_vary[idx])))
plt.title(r'$dP_g/dT_{ncdm}$ for $z=0.7$, $m_\chi=1.0$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(t_vary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.T_ncdm_var[0]): 
    plt.plot(k_table, ps.dps_table['T_ncdm']*dT_chi_domega_chi, label=r'$\delta T_{ncdm}=$'+'{0:.2f}'.format((t_vary[idx])))
plt.title(r'$dP_g/domega_{ncdm}$ for $z=0.7$, $m_\chi=1.0$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

In [ ]:
#Plot derivatives of power spectra
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.h_var[0]): 
    plt.plot(k_table, ps.log_dps_table['h'], label=r'$\delta$'+'h={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dlogP_g/dh$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_b_var[0]): 
    plt.plot(k_table, ps.log_dps_table['omega_b'], label=r'$\delta\omega_b=$'+'{0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dlogP_g/d\omega_b$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.omega_cdm_var[0]): 
    plt.plot(k_table, ps.log_dps_table['omega_cdm'], label=r'$\delta\omega_{cdm}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dlogP_g/d\omega_{cdm}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.tau_reio_var[0]): 
    plt.plot(k_table, ps.log_dps_table['tau_reio'], label=r'$\delta\tau_{reio}$'+'={0:.2f}%'.format((relvary[idx]-1)*100.))
plt.title(r'$dlogP_g/d\tau_{reio}$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(t_vary))
plt.figure(figsize=(10,10))
for idx, ps in enumerate(a1.T_ncdm_var[0]): 
    plt.plot(k_table, ps.log_dps_table['T_ncdm'], label=r'$\delta T_{ncdm}=$'+'{0:.2f}'.format((t_vary[idx])))
plt.title(r'$dlogP_g/dT_{ncdm}$ for $z=0.7$, $m_\chi=1.0$ eV')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

In [ ]:
#Plot differences in derivatives of power spectra
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.h_var[0])-1): 
    plt.plot(k_table, a1.h_var[0][i+1].dps_table['h'] - a1.h_var[0][i].dps_table['h'], 
             label=r'$\Delta$ h $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/dh)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_b_var[0])-1): 
    plt.plot(k_table, a1.omega_b_var[0][i+1].dps_table['omega_b'] - a1.omega_b_var[0][i].dps_table['omega_b'], 
             label=r'$\Delta \omega_b |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\omega_b)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_cdm_var[0])-1): 
    plt.plot(k_table, a1.omega_cdm_var[0][i+1].dps_table['omega_cdm'] - a1.omega_cdm_var[0][i].dps_table['omega_cdm'], 
             label=r'$\Delta \omega_cdm |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\omega_cdm)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.tau_reio_var[0])-1): 
    plt.plot(k_table, a1.tau_reio_var[0][i+1].dps_table['tau_reio'] - a1.tau_reio_var[0][i].dps_table['tau_reio'], 
             label=r'$\Delta \tau_{reio}$'+' $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dP_g/d\tau_{reio})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.T_ncdm_var[0])-1): 
    plt.plot(k_table, a1.T_ncdm_var[0][i+1].dps_table['T_ncdm'] - a1.T_ncdm_var[0][i].dps_table['T_ncdm'], 
             label=r'$\Delta T_{ncdm}$'+' $|^{:.1f}_{:.1f}$'.format(t_vary[i+1], t_vary[i]))
plt.title(r'$\Delta(dP_g/dT_{ncdm})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

In [ ]:
#Plot differences in derivatives of power spectra
sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.h_var[0])-1): 
    plt.plot(k_table, a1.h_var[0][i+1].log_dps_table['h'] - a1.h_var[0][i].log_dps_table['h'], 
             label=r'$\Delta$ h $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dlogP_g/dh)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_b_var[0])-1): 
    plt.plot(k_table, a1.omega_b_var[0][i+1].log_dps_table['omega_b'] - a1.omega_b_var[0][i].log_dps_table['omega_b'], 
             label=r'$\Delta \omega_b |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dlogP_g/d\omega_b)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.omega_cdm_var[0])-1): 
    plt.plot(k_table, a1.omega_cdm_var[0][i+1].log_dps_table['omega_cdm'] - a1.omega_cdm_var[0][i].log_dps_table['omega_cdm'], 
             label=r'$\Delta \omega_cdm |^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dlogP_g/d\omega_cdm)$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.tau_reio_var[0])-1): 
    plt.plot(k_table, a1.tau_reio_var[0][i+1].log_dps_table['tau_reio'] - a1.tau_reio_var[0][i].log_dps_table['tau_reio'], 
             label=r'$\Delta \tau_{reio}$'+' $|^{:.0f}_{:.0f}$ %'.format((relvary[i+1]-1)*100., (relvary[i]-1)*100.))
plt.title(r'$\Delta(dlogP_g/d\tau_{reio})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()

sns.set_palette("Blues_d", n_colors=len(relvary))
plt.figure(figsize=(10,10))
for i in range(len(a1.T_ncdm_var[0])-1): 
    plt.plot(k_table, a1.T_ncdm_var[0][i+1].log_dps_table['T_ncdm'] - a1.T_ncdm_var[0][i].log_dps_table['T_ncdm'], 
             label=r'$\Delta T_{ncdm}$'+' $|^{:.1f}_{:.1f}$'.format(t_vary[i+1], t_vary[i]))
plt.title(r'$\Delta(dlogP_g/dT_{ncdm})$ for $z=0.7$')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.legend()
plt.show()